In [1]:
import pandas as pd
import io

# Simulate reading the CSV data from the provided strings
# In a real pipeline, you would replace io.StringIO(csv_data) with a file path:
# pd.read_csv('s3://your-bucket/employee_master.csv') or pd.read_csv('employee_master.csv')

# --- 1. Employee Master Data ---
master_csv = """Employee_ID,Hire_Date,Job_Role,Job_Level,Monthly_Base_Salary,Termination_Date
E001,2021-03-01,Sales Executive,3,5500,
E002,2017-06-01,Senior Dev,4,11200,
E003,2023-11-01,Marketing Analyst,2,4100,
E004,2018-01-15,HR Generalist,3,6000,2025-08-01
E005,2024-09-15,Junior Dev,2,8500,
E006,2022-04-10,Financial Manager,4,9800,
E007,2023-07-01,Sales Lead,3,7200,2025-09-30
"""
df_master = pd.read_csv(io.StringIO(master_csv), parse_dates=['Hire_Date', 'Termination_Date'])

In [2]:
# --- 2. Promotion History ---
promo_csv = """Employee_ID,Promotion_Date,New_Job_Level
E001,2022-01-01,3
E002,2024-07-01,4
E003,2024-03-01,2
E004,2020-07-01,3
E006,2022-04-10,4
E006,2024-01-01,4
E007,2024-01-01,3
"""
df_promo = pd.read_csv(io.StringIO(promo_csv), parse_dates=['Promotion_Date'])

In [3]:
# --- 3. Timesheet Data ---
ot_csv = """Employee_ID,Total_OT_Hours_90_Days
E001,135
E002,45
E003,0
E004,180
E005,18
E006,108
E007,270
"""
df_ot = pd.read_csv(io.StringIO(ot_csv))

In [4]:
# --- 4. Performance Data ---
perf_csv = """Employee_ID,Review_Date,Rating_Score
E001,2025-01-01,3.5
E002,2025-03-01,4.5
E003,2024-12-01,3.0
E004,2024-08-01,2.5
E005,2024-12-01,3.8
E006,2025-01-01,4.0
E007,2024-12-01,3.5
"""
df_perf = pd.read_csv(io.StringIO(perf_csv), parse_dates=['Review_Date'])

print("All four DataFrames are loaded and ready for transformation.")

All four DataFrames are loaded and ready for transformation.


In [7]:
import pandas as pd
from sqlalchemy import create_engine
import io

# --- A. Database Credentials ---
# IMPORTANT: Replace these with your actual credentials
DB_USER = "postgres"        
DB_PASSWORD = "postgres"    
DB_HOST = "localhost"            
DB_PORT = "5433"                 
DB_NAME = "hr_data"             

# SQLAlchemy connection string
DB_STRING = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# --- C. Define Loading Instructions ---
# Pair the loaded DataFrames with their intended PostgreSQL table names
dataframes_to_load = [
    {"df": df_master, "table_name": "employee_master"},
    {"df": df_promo, "table_name": "promotion_history"},
    {"df": df_ot, "table_name": "timesheet_data"},
    {"df": df_perf, "table_name": "performance_data"}
]

# --- D. Execution Loop ---
try:
    # Create the connection engine
    engine = create_engine(DB_STRING)
    
    print("Starting DataFrame load to PostgreSQL...")

    for item in dataframes_to_load:
        df = item['df']
        table_name = item['table_name']

        # Use .to_sql() to write the DataFrame directly to the database
        df.to_sql(
            name=table_name,
            con=engine,
            if_exists='replace', # Overwrite the table if it already exists
            index=False          # Do not write the DataFrame index as a column
        )
        print(f"   ✅ Loaded {len(df)} rows into table: '{table_name}'")

    print("\nAll four source tables successfully transferred to PostgreSQL.")

except Exception as e:
    print(f"\n❌ Error during database load: {e}")

finally:
    # Ensure the database connection is closed
    if 'engine' in locals() and engine:
        engine.dispose()

Starting DataFrame load to PostgreSQL...
   ✅ Loaded 7 rows into table: 'employee_master'
   ✅ Loaded 7 rows into table: 'promotion_history'
   ✅ Loaded 7 rows into table: 'timesheet_data'
   ✅ Loaded 7 rows into table: 'performance_data'

All four source tables successfully transferred to PostgreSQL.
